In [9]:
import pandas as pd

df = pd.read_csv("./letterboxd_films.csv")

df

,name,rating,genres,avg_user_rating
0,a-complete-unknown,★★★½,"Drama, Music",3.6
1,bonjour-tristesse-2024,★★★★,"Romance, Drama",3.3
2,alien-romulus,★★,"Science Fiction, Horror",3.6
3,blur-to-the-end,★★★,"Documentary, Music",3.8
4,one-day-2024,★★★★,Drama,4.1
...,...,...,...,...
171,purple-noon,★★★,"Drama, Crime, Thriller",4.0
172,1984,★★★,"Drama, Science Fiction",3.2
173,rebel-without-a-cause,No Rating,Drama,3.8
174,summer-interlude,★★★★,"Drama, Romance",3.8


In [10]:
# Convert the rating to a numerical format
rating_map = {
    '½': 0.5,
    '★': 1,
    '★½': 1.5,
    '★★': 2,
    '★★½': 2.5,
    '★★★': 3,
    '★★★½': 3.5,
    '★★★★': 4,
    '★★★★½': 4.5,
    '★★★★★': 5
}
df['Rating'] = df['rating'].map(rating_map)

In [ ]:
df = df.drop(columns=["genres","rating","avg_user_rating"])


,name,Rating
0,a-complete-unknown,3.5
1,bonjour-tristesse-2024,4.0
2,alien-romulus,2.0
3,blur-to-the-end,3.0
4,one-day-2024,4.0
...,...,...
171,purple-noon,3.0
172,1984,3.0
173,rebel-without-a-cause,NaN
174,summer-interlude,4.0


In [12]:
df.to_csv("liv_ratings")